In [ ]:
import json
from requests import request
from requests.compat import urljoin, urlparse
from requests.exceptions import HTTPError
from urllib.robotparser import RobotFileParser
from bs4 import BeautifulSoup
from time import sleep
import re
from requests.compat import urlparse, urljoin
import sqlite3
from requests import Session
from datetime import datetime, timedelta 

In [ ]:
def canfetch(url, agent='*', path='/'):
    robot = RobotFileParser(urljoin(url, '/robots.txt'))
    robot.read()
    return robot.can_fetch(agent, urlparse(url)[2])
    
def download(url, params={}, headers={}, method='GET', limit=3):
    try:
        session = Session()
        resp = session.request(method, url,
                               params = params if method.upper() == 'GET' else '',
                               data = params if method.upper() == 'POST' else '',
                               headers = headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500:
            print(limit)
            time.sleep(60) # Server Error이기 때문에 delay를 두고 실행하기 위해서 사용한다.
            # 보통은 5분에 한 번꼴로 random하게 되도록 설정한다.
            download(url, params, headers, method, limit-1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.reason)
            print(e.response.headers)
    return resp

In [ ]:
conn = sqlite3.connect('news3.db')
cur = conn.cursor()
cur.executescript('''
    DROP TABLE IF EXISTS head;
    CREATE TABLE head(
        pk INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        head TEXT NOT NULL,
        cdate TEXT NOT NULL,
        wdate TEXT NOT NULL,
        ref INTEGER NOT NULL,
        page INTEGER NOT NULL
    );
''')

cur.executescript('''
    DROP TABLE IF EXISTS history;
    CREATE TABLE history(
        pk INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        seen TEXT NOT NULL,
        ref INTEGER NOT NULL
    );
''')

In [ ]:
seen = list()
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'}

# 시간변수 만들기, 시작날짜 종료날짜 정하기
start_date = datetime.strptime('20190817', '%Y%m%d') 
end_date = datetime.strptime('20200817', '%Y%m%d') 

str_date_list = [] 
while start_date.strftime('%Y%m%d') != end_date.strftime('%Y%m%d'): 
    str_date_list.append(start_date.strftime('%Y%m%d')) 
    start_date += timedelta(days=1)

url = 'https://news.sbs.co.kr/news/newsflash.do?pageDate=' #주소 합치는거 고치기
urls = list()
for i in str_date_list:
    for k in range(40):
        u = url + i +'&pageIdx='+str(k+1)
        urls.append(u)  

count = 0
    
while urls:
    try:
        count += 1
        seed = urls.pop(0)

        resp = download(seed, headers=headers)
        dom = BeautifulSoup(resp.content.decode('utf-8','replace'), 'html.parser')
        seen.append(seed)
        if len(dom.select('#container > div > div.w_news_list.type_issue > ul > li > a.news > p > span.date')) != 0:
            cur.execute('''
                INSERT INTO history(seen, ref) VALUES(?, 304)
            ''', [seed])

        
        if len(dom.select('#container > div > div.w_news_list.type_issue > ul > li > a.news > p > span.date')) != 0:
            head = [_.text.strip() for _ in dom.select('#container > div > div.w_news_list.type_issue > ul > li > a.news > p > strong')]
            wdate = [_.text.strip() for _ in dom.select('#container > div > div.w_news_list.type_issue > ul > li > a.news > p > span.date')]
            cdate = str(datetime.now()).split('.')[0]
            page = re.search('pageIdx=(\d+)', urlparse(seed).query).group(1)
            if len(head)==len(wdate):
                for _ in range(0,len(head)):
                    cur.execute('''
                        INSERT INTO head(head, cdate, wdate, page, ref) VALUES(?,?,?,?,304)
                        ''', [head[_], wdate[_], cdate, page])
                    conn.commit()
        else:
            date = re.search('pageDate=(\d+)', urlparse(seed).query).group(1)
            while date == re.search('pageDate=(\d+)', urlparse(urls[0]).query).group(1):
                urls.pop(0)

        if count%100 == 0:
                print(count)
     
    except Exception as e:
            print('Error',e)